This notebook is about downloading network shapefile using osmnx.

You can also test with the data already downloaded, which is provided at `data/stockholm/data.tar.gz`.

Extract it as 

```bash
cd data/stockholm/
tar -xvfz data.tar.gz
```


In [6]:
import osmnx as ox
import time
from shapely.geometry import Polygon
from shapely.wkt import loads
import os
def stringify_nonnumeric_cols(gdf):
    import pandas as pd
    for col in (c for c in gdf.columns if not c == "geometry"):
        if not pd.api.types.is_numeric_dtype(gdf[col]):
            gdf[col] = gdf[col].fillna("").astype(str)
    return gdf
def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    if 'utils_graph' in dir(ox):        
        gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    elif 'utils' in dir(ox):
        gdf_nodes, gdf_edges = ox.utils.graph_to_gdfs(G)
    else:
        print("Error, graph to gdf function not found")
    gdf_nodes = stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = gdf_edges.index
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

osmnx version 0.10


### Download data in polygon from wkt

In [7]:
polygon_wkt = "POLYGON((17.967294873400338 59.3820100171582,18.1520025150019 59.3820100171582,18.1520025150019 59.28710430777261,17.967294873400338 59.28710430777261,17.967294873400338 59.3820100171582))"
boundary_polygon = loads(polygon_wkt)
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='data/stockholm')
print("--- %s seconds ---" % (time.time() - start_time))

--- 5.348304033279419 seconds ---


### Download network in a boundingbox

In [ ]:
bounds = (18.029122582902115, 18.070836297501724, 59.33476653724975, 59.352622230576124)
x1,x2,y1,y2 = bounds
boundary_polygon = Polygon([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='data/stockholm')
print("--- %s seconds ---" % (time.time() - start_time))

### Download network in a city

In [ ]:
place ="Stockholm, Sweden"
G = ox.graph_from_place(place, network_type='drive', which_result=2)
save_graph_shapefile_directional(G, filepath='stockholm')

### Download network with a boundary in GeoJSON file

In [ ]:
# Download by a boundary polygon in geojson
import osmnx as ox
from shapely.geometry import shape
json_file = open("stockholm_boundary.geojson")
import json
data = json.load(json_file)
boundary_polygon = shape(data["features"][0]['geometry'])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
save_graph_shapefile_directional(G, filepath='stockholm')